## Importing packages

In [36]:
import numpy as np
import pandas as pd

In [62]:
df = pd.read_excel("./ToyotaCorolla.xls", "data")
df.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,1,0,0,0,1,0,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,0,1,0,1,0,0,0,0


## One-hot Encoding


In [63]:
one_hot_columns = ["Fuel_Type", "Color"]
for column in one_hot_columns:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns] = dummies
df.drop(columns=one_hot_columns, inplace=True)

In [64]:
df.shape

(1436, 50)

In [65]:
def train_validation_test_split(df, train_size, validation_size, test_size):
    train = df.sample(frac=train_size, random_state=0)
    df.drop(train.index, inplace=True)
    print("train_size:", train.shape)
    validation_size = (validation_size) / (validation_size + test_size)
    validation = df.sample(frac=validation_size, random_state=0)
    df.drop(validation.index, inplace=True)
    print("validation_size:", validation.shape)
    test = df
    print("test_size:", test.shape)
    return train, validation, test

In [66]:
train_size, validation_size, test_size = 0.7, 0.15, 0.15
train, validation, test = train_validation_test_split(df.copy(), train_size, validation_size, test_size)

train_size: (1005, 50)
validation_size: (216, 50)
test_size: (215, 50)
